In [1]:
import torch
import numpy as np

In [2]:
print(torch.__version__)
print("CUDA available:", torch.cuda.is_available())

2.7.1+cu118
CUDA available: True


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [4]:
from torch import nn
import matplotlib.pyplot as plt

In [5]:
import matplotlib
print(matplotlib.__version__)

3.10.8


In [6]:
weight = 0.7
bias = 0.3

start = 0 
end = 1
step = 0.02
X = torch.arange(start,end,step).unsqueeze(dim=1)
y = weight * X + bias

In [7]:
train_split = int(0.8 * len(X))
X_train , y_train = X[:train_split] , y[:train_split]
X_test , y_test = X[train_split:] , y[train_split:]

In [8]:
def plot_predictions(train_data = X_train,
                     train_labels = y_train,
                     test_data = X_test,
                     test_labels = y_test,
                     predictions = None):
    plt.figure(figsize=(10,7))

    plt.scatter(train_data , train_labels , c = "b" , s = 4 , label="Training Data")
    plt.scatter(test_data , test_labels , c = "g" , s = 4 , label="Testing Data")

    if predictions is not None:
        plt.scatter(test_data , predictions , c = "r" , s = 4 , label="Prediction Data")

In [9]:
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.weights = nn.Parameter(torch.randn(1))
        self.bias = nn.Parameter(torch.randn(1))

    def forward(self,x:torch.Tensor):
        return self.weights * x + self.bias 

In [566]:
torch.manual_seed(67)
model_0 = LinearRegressionModel()

with torch.inference_mode():
    y_preds = model_0(X_test)

# plot_predictions(predictions=y_preds)

In [16]:
loss_fn = nn.L1Loss()
optimizer = torch.optim.SGD(params = model_0.parameters(),lr=0.01)

In [567]:
epochs = 10000

for epoch in range(epochs):
    model_0.train()
    y_pred = model_0(X_train)
    loss = loss_fn(y_pred,y_train)
    # print(f"Loss: {loss}")
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

model_0.eval()
with torch.inference_mode():
    y_pred = model_0(X_test)

# plot_predictions(predictions=y_pred)
model_0.state_dict()

OrderedDict([('weights', tensor([1.9416])), ('bias', tensor([-1.3425]))])

In [560]:
from pathlib import Path

MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True,exist_ok=True)

MODEL_NAME = "linear_regression_model.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

torch.save(obj=model_0.state_dict(),f=MODEL_SAVE_PATH)

In [561]:
model_1 = LinearRegressionModel()
model_1.load_state_dict(torch.load(MODEL_SAVE_PATH))

<All keys matched successfully>

In [568]:
model_1.eval()
with torch.inference_mode():
    y_pred_2 = model_1(X_test)

# plot_predictions(predictions=y_pred_2)